In [1]:
import numpy as np
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh
import networkx as nx
import utils

import warnings
warnings.filterwarnings('ignore')

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

Using backend: pytorch


In [2]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCNLayer(1433, 16)
        self.gcn2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.gcn1(g, features))
        x = self.gcn2(g, x)
        return x
net = Net()
print(net)

Net(
  (gcn1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (gcn2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [3]:
def load_cora_data():
    data = citegrh.load_cora()
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    train_mask = torch.BoolTensor(data.train_mask)
    test_mask = torch.BoolTensor(data.test_mask)
    g = data.graph
    # add self loop
    g.remove_edges_from(nx.selfloop_edges(g))
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    return g, features, labels, train_mask, test_mask

In [28]:
def load_gx_data():
    shapelets = np.load('./datamoney_laundrying_dataset/gx_shapelets_362_7.npu', allow_pickle=True)
    
    return graph, feat, labels, train_mask, test_mask

In [2]:
#amount shapelets
# tmp_shps = []
shapelets = np.load('./data/money_laundrying_dataset/gx_amt_shapelets_362_7.npy', allow_pickle=True)
s = shapelets
# for shp in shapelets:
#     tmp_shps.append(shp)
# shapelets = np.array(tmp_shps).squeeze()
# shapelets.shape

(7, 362)

In [3]:
dict_ts = utils.load_ts()

In [4]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [21]:
g, features, labels, train_mask, test_mask = load_cora_data()

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, pl

In [23]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Epoch 00000 | Loss 1.9661 | Test Acc 0.2700 | Time(s) nan
Epoch 00001 | Loss 1.9357 | Test Acc 0.3540 | Time(s) nan
Epoch 00002 | Loss 1.9077 | Test Acc 0.3800 | Time(s) nan
Epoch 00003 | Loss 1.8816 | Test Acc 0.4150 | Time(s) 0.0075


/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, pl

Epoch 00004 | Loss 1.8555 | Test Acc 0.4440 | Time(s) 0.0080
Epoch 00005 | Loss 1.8295 | Test Acc 0.4620 | Time(s) 0.0079
Epoch 00006 | Loss 1.8055 | Test Acc 0.4870 | Time(s) 0.0078
Epoch 00007 | Loss 1.7821 | Test Acc 0.5040 | Time(s) 0.0078
Epoch 00008 | Loss 1.7593 | Test Acc 0.5110 | Time(s) 0.0077
Epoch 00009 | Loss 1.7373 | Test Acc 0.5250 | Time(s) 0.0076
Epoch 00010 | Loss 1.7162 | Test Acc 0.5330 | Time(s) 0.0075
Epoch 00011 | Loss 1.6960 | Test Acc 0.5440 | Time(s) 0.0074
Epoch 00012 | Loss 1.6766 | Test Acc 0.5490 | Time(s) 0.0074
Epoch 00013 | Loss 1.6579 | Test Acc 0.5550 | Time(s) 0.0073
Epoch 00014 | Loss 1.6397 | Test Acc 0.5650 | Time(s) 0.0073
Epoch 00015 | Loss 1.6220 | Test Acc 0.5760 | Time(s) 0.0072
Epoch 00016 | Loss 1.6049 | Test Acc 0.5810 | Time(s) 0.0072
Epoch 00017 | Loss 1.5883 | Test Acc 0.5850 | Time(s) 0.0072
Epoch 00018 | Loss 1.5722 | Test Acc 0.5940 | Time(s) 0.0071
Epoch 00019 | Loss 1.5565 | Test Acc 0.6000 | Time(s) 0.0071
Epoch 00020 | Loss 1.541

In [31]:
print(g)

Graph(num_nodes=2708, num_edges=13264,
      ndata_schemes={}
      edata_schemes={})


In [37]:
print(features.shape)
print(features)

torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [45]:
labels

tensor([3, 4, 4,  ..., 3, 3, 3])

In [46]:
print(train_mask.shape)

torch.Size([2708])


In [48]:
print(test_mask.shape)

torch.Size([2708])


In [101]:
\

<module 'dgl.data.citation_graph' from '/home/amax/anaconda3/lib/python3.8/site-packages/dgl/data/citation_graph.py'>

In [52]:
data()

TypeError: 'CoraGraphDataset' object is not callable